# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [86]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from Vacation_API import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
#Load CityWeather df
city_weather_df = pd.read_csv('../WeatherPy/UpgradedCityWeather.csv')
del city_weather_df['Unnamed: 0']
city_weather_df.head()

,City,Latitude,Longitude,Max_Temps,Humidity,Cloudiness,Wind_Speed,Country,Date,Max_Temps_in_Fahrenheit,Imperial_Wind_Speed
0,Mackay,-21.1500,149.2000,297.62,35,4,4.12,AU,1623981388,76.05,9.23
1,Tromsø,69.6496,18.9570,282.51,93,75,0.00,NO,1623981391,48.85,0.00
2,Mataura,-46.1927,168.8643,281.19,75,7,3.24,NZ,1623981393,46.47,7.26
3,Chapleau,47.8405,-83.4034,295.20,49,75,3.09,CA,1623981396,71.69,6.92
4,Longyearbyen,78.2186,15.6401,277.06,65,75,5.66,SJ,1623981399,39.04,12.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Storing latitude and longitude in locations
locations = city_weather_df[['Latitude', 'Longitude']]

humidity = city_weather_df['Humidity']

#Plot Heatmap
fig = gmaps.figure()

#Creating heat leayer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                max_intensity=100, point_radius=1)

#add layer
fig.add_layer(heat_layer)

#Display Figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [66]:
f_filter_df = city_weather_df[(city_weather_df['Max_Temps_in_Fahrenheit'] >=85) & 
                (city_weather_df['Max_Temps_in_Fahrenheit'] <=100)]
s_filter_df = f_filter_df[f_filter_df['Imperial_Wind_Speed']<=15]
narrowed_df = s_filter_df[s_filter_df['Cloudiness']<=30]
narrowed_df

,City,Latitude,Longitude,Max_Temps,Humidity,Cloudiness,Wind_Speed,Country,Date,Max_Temps_in_Fahrenheit,Imperial_Wind_Speed
66,Lompoc,34.6391,-120.4579,304.23,72,1,5.14,US,1623981246,87.94,11.51
101,Bilma,18.6853,12.9164,304.83,11,0,2.37,NE,1623981641,89.02,5.31
107,Arlit,18.7369,7.3853,304.87,18,6,3.77,NE,1623981657,89.10,8.44
128,Douentza,14.9951,-2.9517,305.51,30,11,5.42,ML,1623981710,90.25,12.14
150,Tessalit,20.1986,1.0114,305.48,11,0,3.13,ML,1623981731,90.19,7.01
171,Innisfail,-17.5333,146.0333,304.19,68,13,3.52,AU,1623981819,87.87,7.88
181,Doha,25.2867,51.5333,306.14,27,0,6.69,QA,1623981598,91.38,14.99
228,Kloulklubed,7.0419,134.2556,303.43,72,21,0.95,PW,1623981964,86.50,2.13
235,Balkanabat,39.5108,54.3671,305.12,12,0,4.37,TM,1623981983,89.55,9.79
274,Aloleng,16.1307,119.7824,302.95,71,21,3.65,PH,1623982082,85.64,8.18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [68]:
#Store df into varible and add
hotel_df = narrowed_df.loc[:,['City','Country','Latitude', 'Longitude']].reset_index(drop=True)
hotel_df['Hotel_Name'] = ''
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel_Name
0,Lompoc,US,34.6391,-120.4579,
1,Bilma,NE,18.6853,12.9164,
2,Arlit,NE,18.7369,7.3853,
3,Douentza,ML,14.9951,-2.9517,
4,Tessalit,ML,20.1986,1.0114,


In [95]:
#Set Parameters to search for hotels using base URL 
#(Referenced: https://developers.google.com/maps/documentation/places/web-service/search)
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"key" : g_key,
          "radius" : 5000,
          "type" : "hotel",
          "keyword" : "hotels"}
drop_index = []
#Start For loop to go through each row and retrieve lodging
for index, row in hotel_df.iterrows():
    city = row['City']
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # add keyword to params dict
    params['location'] = f"{latitude},{longitude}"
    
    # assemble url and make API request
    print(f'Retreiving Resultsfor Index {index}: {city}.')
    response = requests.get(base_url, params=params).json()
    
    #Extract Results
    results = response['results']
    
    try:
        print(f"Closest lodging in {city} is {results[0]['name']}.")
        if results[0]['name'] != '':
            hotel_df.loc[index,'Hotel_Name'] = results[0]['name']
    except (KeyError, IndexError):
        drop_index.append(index)
        print('Missing field/result...skipping.')
    print('-'*10)
    time.sleep(1)
print(f'-----End of Search-----')

Retreiving Resultsfor Index 0: Lompoc.
Closest lodging in Lompoc is Embassy Suites by Hilton Lompoc Central Coast.
----------
Retreiving Resultsfor Index 1: Bilma.
Missing field/result...skipping.
----------
Retreiving Resultsfor Index 2: Arlit.
Closest lodging in Arlit is Hôtel Temet.
----------
Retreiving Resultsfor Index 3: Douentza.
Closest lodging in Douentza is Ogomono Doumbo.
----------
Retreiving Resultsfor Index 4: Tessalit.
Missing field/result...skipping.
----------
Retreiving Resultsfor Index 5: Innisfail.
Closest lodging in Innisfail is Tropical Lodge.
----------
Retreiving Resultsfor Index 6: Doha.
Closest lodging in Doha is Sharq Village & Spa, a Ritz-Carlton Hotel.
----------
Retreiving Resultsfor Index 7: Kloulklubed.
Closest lodging in Kloulklubed is Carp Island Resort.
----------
Retreiving Resultsfor Index 8: Balkanabat.
Closest lodging in Balkanabat is Nebitchi Hotel.
----------
Retreiving Resultsfor Index 9: Aloleng.
Missing field/result...skipping.
----------
Ret

In [109]:
results

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 33.1299444, 'lng': -107.2561083},
   'viewport': {'northeast': {'lat': 33.13103577989272,
     'lng': -107.2547547201073},
    'southwest': {'lat': 33.12833612010728, 'lng': -107.2574543798927}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'name': 'Sierra Grande Lodge & Spa',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 3456,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103544069589107987021">Fernando Valdez</a>'],
    'photo_reference': 'ATtYBwJXBf95wrEjKxcUwjonH4xDH8GXzrVcxUuuOVF6ENUnF_hN545i8WTvHSyQx0d2VoRQas1nEV7Q5WAj_zxbyGinO2n1nmnF93rfj7M4lX7ZlSjcZf-EeNSHM4YxgCSP1yT6l4hDpGnno4O63Rku2wXFmlRVmWIpZgdts0mls7mAzkXK',
    'width': 4608}],
  'place_id': 'ChIJGW1vWcOC34YR3pr4c36Ij-o',
  'plus_code': {'compound_code': '4PHV+XH Truth or Consequences, New Mexico',
   'global_code': '855J4PHV+XH'},
  'rating': 4.6,
  'reference': 

In [96]:
drop_index

[1, 4, 9]

In [100]:
clean_hotel_df = hotel_df.drop(drop_index).reset_index(drop=True)
clean_hotel_df

,City,Country,Latitude,Longitude,Hotel_Name
0,Lompoc,US,34.6391,-120.4579,Embassy Suites by Hilton Lompoc Central Coast
1,Arlit,NE,18.7369,7.3853,Hôtel Temet
2,Douentza,ML,14.9951,-2.9517,Ogomono Doumbo
3,Innisfail,AU,-17.5333,146.0333,Tropical Lodge
4,Doha,QA,25.2867,51.5333,"Sharq Village & Spa, a Ritz-Carlton Hotel"
5,Kloulklubed,PW,7.0419,134.2556,Carp Island Resort
6,Balkanabat,TM,39.5108,54.3671,Nebitchi Hotel
7,Atar,MR,20.5169,-13.0499,Bab Sahara
8,Platteville,US,42.7342,-90.4785,"Holiday Inn Express & Suites Platteville, an I..."
9,Qui Nhon,VN,13.7667,109.2333,Khách sạn Hải Âu Quy Nhơn - Seagull Hotel


In [103]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Latitude", "Longitude"]]

In [108]:
#Creation of Marker Layer
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info, display_info_box=True)

# Add marker layer ontop of heat map
fig.add_layer(marker_layer)

#Display Figure
fig

Figure(layout=FigureLayout(height='420px'))